# newer crf trained with kfolded data

In [1]:
#Data Analysis
import pandas as pd
import numpy as np

#Data Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
sns.set(font_scale=1)
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

from copy import deepcopy
from collections import defaultdict

#Modeling
import fasttext
from sklearn.model_selection import cross_val_predict, cross_val_score, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn_crfsuite import CRF, scorers, metrics
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.metrics import classification_report, make_scorer
import scipy.stats
import eli5

#ner_eval
# import sys
# sys.path.append('././')
from ner_eval import collect_named_entities
from ner_eval import compute_metrics
from ner_eval import compute_precision_recall_wrapper

# import file

In [2]:
path = "C:\\Users\\farih\\Downloads\\"
indoNLUmodel = "C:\\Users\\farih\\Downloads\\fasttext.4B.id.300.epoch5.uncased.bin"
model = fasttext.load_model(indoNLUmodel)

In [5]:
train_df = pd.read_csv("D:\Drive\KULEEAH\Semester 8\TA Farihin\Dataset\\StratifiedKFold\\trainval.csv")
test_df = pd.read_csv("D:\Drive\KULEEAH\Semester 8\TA Farihin\Dataset\\StratifiedKFold\\test.csv")

In [6]:
train_df.head(1)

,tweet_id,token,ner,token_idx,ner_idx,entities,count_entities,onehot_entities
0,1380681275594305537,"['@ridwankamil', '@KickAndyShow', '@Metro_TV',...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","[17441, 14696, 23089, 29865, 5288, 16801, 4219...","[12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 1...","[['gubernur', 'PER'], ['jabar', 'LOC']]","[Counter({'PER': 1, 'LOC': 1})]",110000


In [7]:
def getsentence(df):
    def _zip(a,b):
            temp_list= list()
            for i in range(len(a)):
                temp_list.append((a[i],b[i]))
            return temp_list
    sentences = list()
    for idx, row in df.iterrows():
        zipped_sentence = tuple(_zip(eval(row.token),eval(row.ner)))
        sentences.append(zipped_sentence)
    return sentences

In [8]:
sentences_train = getsentence(train_df)
sentences_test = getsentence(test_df)
print(sentences_train[:1])
print(sentences_test[:1])


[(('@ridwankamil', 'O'), ('@KickAndyShow', 'O'), ('@Metro_TV', 'O'), ('Upaya', 'O'), ('untuk', 'O'), ('mnaikan', 'O'), ('nilai', 'O'), ('jual', 'O'), ('yg', 'O'), ('bgus,dngn', 'O'), ('cara', 'O'), ('d', 'O'), ('desain', 'O'), ('pak', 'O'), ('gubernur', 'B-PER'), ('akan', 'O'), ('menjadikn', 'O'), ('produk', 'O'), ('umkm', 'O'), ('d', 'O'), ('mintai', 'O'), ('krena', 'O'), ('yg', 'O'), ('desain', 'O'), ('org', 'O'), ('nomer', 'O'), ('satu', 'O'), ('di', 'O'), ('jabar', 'B-LOC'))]
[(('@GyuuPotter', 'O'), ('iyhh', 'O'), (',', 'O'), ('soalnya', 'O'), ('mwu', 'O'), ('kerjain', 'O'), ('tugas', 'O'), ('jg��', 'O'))]


In [9]:
# words = list(train_df["token"]))
# n_words = len(words)
# print(n_words)

In [10]:

# Feature set
def word2features(sent, i):
    word = sent[i][0]
    
    features = {
        'bias': 1.0,
        'word.lower()': word.lower() if type(word) == str else word,
        'word[-3:]': word[-3:] if type(word) == str else word,
        'word[-2:]': word[-2:] if type(word) == str else word,
        'word[-1:]': word[-2:] if type(word) == str else word,
        'word[:3]': word[:3] if type(word) == str else word,
        'word[:2]': word[:2] if type(word) == str else word,
        'word[:1]': word[:2] if type(word) == str else word,
        'word.isupper()': word.isupper() if type(word) == str else word,
        'word.istitle()': word.istitle() if type(word) == str else word,
        'word.isdigit()': word.isdigit() if type(word) == str else word,
    }
    if i > 0:
        word1 = sent[i-1][0]
        features.update({
            '-1:word.lower()': word1.lower() if type(word1) == str else word1,
            '-1:word.istitle()': word1.istitle() if type(word1) == str else word1,
            '-1:word.isupper()': word1.isupper() if type(word1) == str else word1,
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        features.update({
            '+1:word.lower()': word1.lower() if type(word1) == str else word1,
            '+1:word.istitle()': word1.istitle() if type(word1) == str else word1,
            '+1:word.isupper()': word1.isupper() if type(word1) == str else word1,
        })
    else:
        features['EOS'] = True

    #src: https://stackoverflow.com/questions/58736548/how-to-use-word-embedding-as-features-for-crf-sklearn-crfsuite-mod   el-training
    we=model.get_word_vector(features['word.lower()'])
    for iv,value in enumerate(we):
        features['v{}'.format(iv)]=value

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

## Feature Extraction

In [11]:
%%time
#Creating the train and test set
X_train = [sent2features(s) for s in sentences_train]
X_test = [sent2features(s) for s in sentences_test]
y_train = [sent2labels(s) for s in sentences_train]
y_test = [sent2labels(s) for s in sentences_test]

Wall time: 2min 14s


In [12]:
# group B and I results
# #First we select all the tags that are relevant for us i.e. remove the 'O' tag from the list. 

# labels = list(crf2.classes_)
# labels = list(train_df['labels'].unique())
# labels = list(filter(lambda a: a != 'O', labels))
# sorted_labels = sorted(
#     labels,
#     key=lambda name: (name[1:], name[0])
# )
sorted_labels=['B-EV', 'I-EV', 'B-LOC', 'I-LOC', 'B-ORG', 'I-ORG', 'B-PER', 'I-PER', 'B-PROD', 'I-PROD', 'B-WA', 'I-WA']
print(sorted_labels)

['B-EV', 'I-EV', 'B-LOC', 'I-LOC', 'B-ORG', 'I-ORG', 'B-PER', 'I-PER', 'B-PROD', 'I-PROD', 'B-WA', 'I-WA']


## Training

In [13]:
%%time
#Now we will create the Randomized CV search model wherein we will use a modified F1 scorer model considering only the relevant labels
# define fixed parameters and parameters to search
crf = CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)   
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=sorted_labels)

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=8, #-1
                        n_iter=50 ,#50,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

d:\Drive\KULEEAH\Semester 8\TA Farihin\env\lib\site-packages\sklearn\base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
d:\Drive\KULEEAH\Semester 8\TA Farihin\env\lib\site-packages\joblib\externals\loky\process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


PicklingError: Could not pickle the task to send it to the workers.

In [ ]:
#Lets check the best estimated parameters and CV score
print('Best parameters:', rs.best_params_)
print('Best CV score:', rs.best_score_)
print('Model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

Best parameters: {'c1': 0.1446123263853422, 'c2': 0.17667994468993134}
Best CV score: 0.6266895273896679
Model size: 2.72M


## Performance per label type per token

In [ ]:
#Now we create the model again using the best estimators
best_crf = rs.best_estimator_
y_pred = best_crf.predict(X_test)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
    ))

d:\Drive\KULEEAH\Semester 8\TA Farihin\env\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass labels=['B-EV', 'I-EV', 'B-LOC', 'I-LOC', 'B-ORG', 'I-ORG', 'B-PER', 'I-PER', 'B-PROD', 'I-PROD', 'B-WA', 'I-WA'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


              precision    recall  f1-score   support

        B-EV      0.639     0.264     0.373       148
        I-EV      0.614     0.352     0.447       199
       B-LOC      0.796     0.686     0.737       437
       I-LOC      0.738     0.546     0.628       196
       B-ORG      0.658     0.440     0.527       389
       I-ORG      0.529     0.513     0.521       193
       B-PER      0.929     0.688     0.791      1148
       I-PER      0.917     0.804     0.857       603
      B-PROD      0.891     0.797     0.842       676
      I-PROD      0.733     0.543     0.624       243
        B-WA      0.667     0.071     0.129        28
        I-WA      0.545     0.128     0.207        47

   micro avg      0.825     0.636     0.718      4307
   macro avg      0.721     0.486     0.557      4307
weighted avg      0.815     0.636     0.709      4307



In [ ]:
best_crf.fit(X_train,y_train)

CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1446123263853422,
    c2=0.17667994468993134, keep_tempfiles=None, max_iterations=100)

In [ ]:
# eli5.show_weights(crf3, top=30)

## Performance over full named-entity

In [16]:
ner_tag = ['LOC', 'PER', 'ORG', 'PROD', 'EV' ,'WA']

In [17]:
test_sents_labels = []
for sentence in sentences_test:
    sentence = [token[1] for token in sentence]
    test_sents_labels.append(sentence)

In [18]:
index = 2
true = collect_named_entities(test_sents_labels[index])
pred = collect_named_entities(y_pred[index])

In [19]:
true

[Entity(e_type='PER', start_offset=4, end_offset=5)]

In [20]:
pred

[Entity(e_type='PER', start_offset=4, end_offset=5)]

In [21]:
compute_metrics(true, pred, ner_tag)

({'strict': {'correct': 1,
   'incorrect': 0,
   'partial': 0,
   'missed': 0,
   'spurious': 0,
   'precision': 0,
   'recall': 0,
   'actual': 1,
   'possible': 1},
  'ent_type': {'correct': 1,
   'incorrect': 0,
   'partial': 0,
   'missed': 0,
   'spurious': 0,
   'precision': 0,
   'recall': 0,
   'actual': 1,
   'possible': 1},
  'partial': {'correct': 1,
   'incorrect': 0,
   'partial': 0,
   'missed': 0,
   'spurious': 0,
   'precision': 0,
   'recall': 0,
   'actual': 1,
   'possible': 1},
  'exact': {'correct': 1,
   'incorrect': 0,
   'partial': 0,
   'missed': 0,
   'spurious': 0,
   'precision': 0,
   'recall': 0,
   'actual': 1,
   'possible': 1}},
 {'LOC': {'strict': {'correct': 0,
    'incorrect': 0,
    'partial': 0,
    'missed': 0,
    'spurious': 0,
    'precision': 0,
    'recall': 0,
    'actual': 0,
    'possible': 0},
   'ent_type': {'correct': 0,
    'incorrect': 0,
    'partial': 0,
    'missed': 0,
    'spurious': 0,
    'precision': 0,
    'recall': 0,
    '

In [23]:
index = 2
true_named_entities_type = defaultdict(list)
pred_named_entities_type = defaultdict(list)

for true in collect_named_entities(test_sents_labels[index]):
    true_named_entities_type[true.e_type].append(true)

for pred in collect_named_entities(y_pred[index]):
    pred_named_entities_type[pred.e_type].append(pred)

In [24]:
true_named_entities_type

defaultdict(list,
            {'PER': [Entity(e_type='PER', start_offset=4, end_offset=5)]})

In [25]:
pred_named_entities_type

defaultdict(list,
            {'PER': [Entity(e_type='PER', start_offset=4, end_offset=5)]})

## Result over all messages

In [26]:
# metrics_results = {'correct': 0, 'incorrect': 0, 'partial': 0,
#                    'missed': 0, 'spurious': 0, 'possible': 0, 'actual': 0, 'precision': 0, 'recall': 0}

# # overall results
# results = {'strict': deepcopy(metrics_results),
#            'ent_type': deepcopy(metrics_results),
#            'partial':deepcopy(metrics_results),
#            'exact':deepcopy(metrics_results)
#           }


# # results aggregated by entity type
# evaluation_agg_entities_type = {e: deepcopy(results) for e in ner_tag}

# for true_ents, pred_ents in zip(test_sents_labels, y_pred):
    
#     # compute results for one message
#     tmp_results, tmp_agg_results = compute_metrics(
#         collect_named_entities(true_ents), collect_named_entities(pred_ents),  ner_tag
#     )
    
#     #print(tmp_results)

#     # aggregate overall results
#     for eval_schema in results.keys():
#         for metric in metrics_results.keys():
#             results[eval_schema][metric] += tmp_results[eval_schema][metric]
            
#     # Calculate global precision and recall
        
#     results = compute_precision_recall_wrapper(results)


#     # aggregate results by entity type
 
#     for e_type in ner_tag:

#         for eval_schema in tmp_agg_results[e_type]:

#             for metric in tmp_agg_results[e_type][eval_schema]:
                
#                 evaluation_agg_entities_type[e_type][eval_schema][metric] += tmp_agg_results[e_type][eval_schema][metric]
                
#         # Calculate precision recall at the individual entity level
                
#         evaluation_agg_entities_type[e_type] = compute_precision_recall_wrapper(evaluation_agg_entities_type[e_type])
                
    

In [27]:
results

{'ent_type': {'correct': 1824,
  'incorrect': 84,
  'partial': 0,
  'missed': 949,
  'spurious': 259,
  'possible': 2857,
  'actual': 2167,
  'precision': 0.8417166589755423,
  'recall': 0.6384319215960798},
 'partial': {'correct': 1841,
  'incorrect': 0,
  'partial': 67,
  'missed': 949,
  'spurious': 259,
  'possible': 2857,
  'actual': 2167,
  'precision': 0.8650207660359944,
  'recall': 0.6561078053902695},
 'strict': {'correct': 1781,
  'incorrect': 127,
  'partial': 0,
  'missed': 949,
  'spurious': 259,
  'possible': 2857,
  'actual': 2167,
  'precision': 0.821873557914167,
  'recall': 0.623381169058453},
 'exact': {'correct': 1841,
  'incorrect': 67,
  'partial': 0,
  'missed': 949,
  'spurious': 259,
  'possible': 2857,
  'actual': 2167,
  'precision': 0.8495616059067835,
  'recall': 0.6443822191109555}}

In [28]:
evaluation_agg_entities_type

{'LOC': {'ent_type': {'correct': 298,
   'incorrect': 14,
   'partial': 0,
   'missed': 130,
   'spurious': 259,
   'possible': 442,
   'actual': 571,
   'precision': 0.521891418563923,
   'recall': 0.6742081447963801},
  'partial': {'correct': 299,
   'incorrect': 0,
   'partial': 13,
   'missed': 130,
   'spurious': 259,
   'possible': 442,
   'actual': 571,
   'precision': 0.5350262697022767,
   'recall': 0.6911764705882353},
  'strict': {'correct': 290,
   'incorrect': 22,
   'partial': 0,
   'missed': 130,
   'spurious': 259,
   'possible': 442,
   'actual': 571,
   'precision': 0.5078809106830122,
   'recall': 0.6561085972850679},
  'exact': {'correct': 299,
   'incorrect': 13,
   'partial': 0,
   'missed': 130,
   'spurious': 259,
   'possible': 442,
   'actual': 571,
   'precision': 0.5236427320490368,
   'recall': 0.6764705882352942}},
 'PER': {'ent_type': {'correct': 780,
   'incorrect': 7,
   'partial': 0,
   'missed': 363,
   'spurious': 259,
   'possible': 1150,
   'actual

In [34]:
from ner_eval import Evaluator

In [35]:
evaluator = Evaluator(test_sents_labels, y_pred, ner_tag)

In [36]:
results, results_agg = evaluator.evaluate()

2021-12-02 17:26:45 root INFO: Imported 1485 predictions for 1485 true examples


In [37]:
results

{'ent_type': {'correct': 1824,
  'incorrect': 84,
  'partial': 0,
  'missed': 949,
  'spurious': 259,
  'possible': 2857,
  'actual': 2167,
  'precision': 0.8417166589755423,
  'recall': 0.6384319215960798},
 'partial': {'correct': 1841,
  'incorrect': 0,
  'partial': 67,
  'missed': 949,
  'spurious': 259,
  'possible': 2857,
  'actual': 2167,
  'precision': 0.8650207660359944,
  'recall': 0.6561078053902695},
 'strict': {'correct': 1781,
  'incorrect': 127,
  'partial': 0,
  'missed': 949,
  'spurious': 259,
  'possible': 2857,
  'actual': 2167,
  'precision': 0.821873557914167,
  'recall': 0.623381169058453},
 'exact': {'correct': 1841,
  'incorrect': 67,
  'partial': 0,
  'missed': 949,
  'spurious': 259,
  'possible': 2857,
  'actual': 2167,
  'precision': 0.8495616059067835,
  'recall': 0.6443822191109555}}

In [38]:
results_agg

{'LOC': {'ent_type': {'correct': 298,
   'incorrect': 14,
   'partial': 0,
   'missed': 130,
   'spurious': 259,
   'possible': 442,
   'actual': 571,
   'precision': 0.521891418563923,
   'recall': 0.6742081447963801},
  'partial': {'correct': 299,
   'incorrect': 0,
   'partial': 13,
   'missed': 130,
   'spurious': 259,
   'possible': 442,
   'actual': 571,
   'precision': 0.5350262697022767,
   'recall': 0.6911764705882353},
  'strict': {'correct': 290,
   'incorrect': 22,
   'partial': 0,
   'missed': 130,
   'spurious': 259,
   'possible': 442,
   'actual': 571,
   'precision': 0.5078809106830122,
   'recall': 0.6561085972850679},
  'exact': {'correct': 299,
   'incorrect': 13,
   'partial': 0,
   'missed': 130,
   'spurious': 259,
   'possible': 442,
   'actual': 571,
   'precision': 0.5236427320490368,
   'recall': 0.6764705882352942}},
 'PER': {'ent_type': {'correct': 780,
   'incorrect': 7,
   'partial': 0,
   'missed': 363,
   'spurious': 259,
   'possible': 1150,
   'actual